<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/langchain/jupyter/autogpt/autogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AutoGPT 智能体

以下`SERPAPI_API_KEY`仅为示例，请访问 https://serpapi.com 注册账号并替换为自己的 `API_KEY`（每月100次免费调用）

In [15]:
!pip install -U langchain \
langchain-openai \
langchain-core \
langchain-community \
google-search-results


  Using cached langchain-0.3.18-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.34-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_community-0.3.17-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain-0.3.18-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.1 MB/s eta 0:00:00
Using cached langchain_core-0.3.34-py3-none-any.whl (412 kB)
Using cached langchain_community-0.3.17-py3-none-any.whl (2.5 MB)
Using cached langchain_text_splitters-0.3.6-py3-none-any.whl (31 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting un

In [16]:
import os

os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"

In [17]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


In [18]:
# 构造 AutoGPT 的工具集
search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [19]:
from langchain_openai import OpenAIEmbeddings

In [21]:
# OpenAI Embedding 模型
# embeddings_model = OpenAIEmbeddings()
# 词嵌入（代理方式）
# OpenAI 词嵌入（代理方式）
embeddings_model = OpenAIEmbeddings(
    api_key="XXX",
    base_url="https://vip.apiyi.com/v1"
)


TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# OpenAI Embedding 向量维数
embedding_size = 1536
# 使用 Faiss 的 IndexFlatL2 索引
index = faiss.IndexFlatL2(embedding_size)
# 实例化 Faiss 向量数据库
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## 实例化自主智能体 Auto-GPT

In [ ]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

In [ ]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model_name="gpt-4", temperature=0, verbose=True),
    memory=vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.8}),# 实例化 Faiss 的 VectorStoreRetriever
)

In [ ]:
# 打印 Auto-GPT 内部的 chain 日志
agent.chain.verbose = True

In [ ]:
agent.run(["2023年成都大运会，中国金牌数是多少"])



> Entering new LLMChain chain...


C:\Users\lenovo\AppData\Roaming\Python\Python310\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年成都大运会，中国金牌数是多少


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description

'I have found and saved the number of gold medals China won in the 2023 Chengdu Universiade.'